# Programmatic Interaction with Recon Server

This notebook demonstrates how to interact with the `server_app.py` programmatically using the `ReconClientApp` class from `client_app.py`.

**Prerequisites:**
1.  The Recon server (`server_app.py`) should be running.
2.  A `recon.opts` file must be configured in the root directory of this project, with at least the `SERVER_HOSTNAME`, `SERVER_PORT`, and `SHARED_KEY` correctly set to match the server's configuration. The `SHARED_KEY` must be identical to the one used by the server.

**Key operations covered:**
*   Setting up dummy input files.
*   Initializing the `ReconClientApp`.
*   Connecting to and disconnecting from the server.
*   Submitting a reconstruction job.
*   Querying the server's status.
*   Viewing the current job queue on the server.

## 1. Setup and Configuration

In [ ]:
import os
import json

# Define paths for dummy input files
dummy_files_dir = "dummy_input_files"
os.makedirs(dummy_files_dir, exist_ok=True)

dummy_file1_path = os.path.join(dummy_files_dir, "P_dummy1.7")
dummy_file2_path = os.path.join(dummy_files_dir, "calibration_dummy.dat")

# Create dummy input files
with open(dummy_file1_path, 'w') as f:
    f.write("This is a dummy PFILE for testing. Actual PFILEs are binary.")

with open(dummy_file2_path, 'w') as f:
    f.write("This is a dummy calibration file.")

print(f"Created dummy file: {dummy_file1_path}")
print(f"Created dummy file: {dummy_file2_path}")

# Example reconstruction options
recon_options_example = {
    "pyscript_name": "slicerecon", # Example: if server has a Python script 'slicerecon.py'
    "num_slices": 128,
    "custom_param": "value_example"
}
print(f"Example recon options: {json.dumps(recon_options_example)}")

## 2. Importing Client Logic

We'll import the `ReconClientApp` from `client_app.py` and `readoptions` from `reconlibs.py`. Ensure this notebook is run from the root of the project so that these modules are found.

In [ ]:
import sys
# Ensure the project root is in the Python path
project_root = os.path.abspath(".") # Assumes notebook is in project root
if project_root not in sys.path:
    sys.path.insert(0, project_root)

try:
    from client_app import ReconClientApp
    from reconlibs import readoptions
    print("Successfully imported ReconClientApp and readoptions.")
except ImportError as e:
    print(f"Error importing modules: {e}\nEnsure the server components are in the PYTHONPATH or run this notebook from the project root directory.")

## 3. Initializing the Client

We initialize `ReconClientApp` by providing the path to the `recon.opts` configuration file. The client will load its connection settings (server hostname, port, shared key) from this file.

In [ ]:
options_file = "recon.opts"

# Check if recon.opts exists, create a default if not (for notebook convenience, user should verify it)
if not os.path.exists(options_file):
    print(f"'{options_file}' not found. Attempting to create a default one.")
    try:
        default_config = readoptions(options_file) # This will create and load defaults
        print(f"Default '{options_file}' created. Please ensure SERVER_HOSTNAME, SERVER_PORT, and especially SHARED_KEY are correctly set to match your server.")
        if default_config.get('SHARED_KEY') == 'SET_YOUR_SHARED_KEY_HERE':
             print("CRITICAL: The SHARED_KEY in the default recon.opts is a placeholder. You MUST change it!")
    except Exception as e:
        print(f"Error creating default '{options_file}': {e}. Please create it manually.")
else:
    print(f"Using existing '{options_file}'.")

client = ReconClientApp(options_file=options_file)
print("ReconClientApp initialized.")

## 4. Demonstrating Client Actions

The `ReconClientApp` has been refactored to provide separate methods for connecting, submitting jobs, querying status, and disconnecting. This allows for more granular programmatic control.

### 4.1 Connect to Server

In [ ]:
if client.connect():
    print("Successfully connected to the server.")
else:
    print("Failed to connect to the server. Ensure the server is running and recon.opts is correct.")

### 4.2 Submit a Job

We'll submit the dummy files created earlier with the example reconstruction options. The server should respond with a Job ID if the submission is successful.

In [ ]:
job_id = None
if client.sf_client and client.sf_client.is_connected():
    files_to_submit = [dummy_file1_path, dummy_file2_path]
    job_id = client.submit_recon_job(files_to_process=files_to_submit, recon_options=recon_options_example)
    if job_id:
        print(f"Job submitted successfully! Job ID: {job_id}")
    else:
        print("Job submission failed.")
else:
    print("Not connected to server. Cannot submit job.")

### 4.3 Check Server Status

This requests general status information from the server.

In [ ]:
if client.sf_client and client.sf_client.is_connected():
    status_info = client.get_server_status()
    if status_info:
        print("\n--- Server Status ---")
        for key, value in status_info.items():
            # Prettify the key for display
            display_key = key.replace('_', ' ').title()
            print(f"{display_key}: {value}")
        print("---------------------\n")
    else:
        print("Failed to retrieve server status.")
else:
    print("Not connected to server.")

### 4.4 View Job Queue

This requests the list of jobs currently in the server's queue.

In [ ]:
if client.sf_client and client.sf_client.is_connected():
    queue_details = client.get_queue_details()
    if queue_details and 'jobs' in queue_details:
        print("\n--- Job Queue ---")
        if queue_details['jobs']:
            for job_info in queue_details['jobs']:
                print(f"  Job ID: {job_info.get('job_id')}")
                print(f"    Status: {job_info.get('status')}")
                print(f"    Primary File: {job_info.get('primary_input_file_name')}")
                print(f"    Num Files: {job_info.get('num_input_files')}")
                print(f"    Submitted: {job_info.get('submitted_at_utc')}")
                print(f"    Server Dir Basename: {job_info.get('job_input_dir_basename')}")
                print("    " + "-"*15)
        else:
            print("Queue is empty.")
        print(f"Total jobs in queue: {queue_details.get('count', 0)}")
        print("-----------------
")
    else:
        print("Failed to retrieve job queue details.")
else:
    print("Not connected to server.")

### 4.5 Disconnect from Server

In [ ]:
client.disconnect()
print("Disconnected from server (if previously connected).")

## 5. Result Handling (Conceptual)

The `ReconClientApp` methods demonstrated above (`submit_recon_job`, `get_server_status`, `get_queue_details`) are for specific, targeted interactions.

For a full job cycle including the download of results, the `client_app.py` script uses a method like `run_full_job_cycle` (which internally calls `submit_recon_job` and then `process_job_and_get_results`). 

The `process_job_and_get_results(job_id)` method would typically:
1.  Listen for server messages related to the given `job_id`.
2.  Handle `dicom_transfer_start` commands by acknowledging readiness.
3.  Receive files sent by the server (e.g., DICOMs) using its internal `SecureFileTransferClient.receive_data()` method, saving them to the directory specified by `CLIENT_DOWNLOAD_DIR` in `recon.opts`.
4.  Wait for a `recon_complete` or `job_failed` message from the server.

To demonstrate this full cycle programmatically using the refactored client, you would call `client.run_full_job_cycle(files_to_process, recon_options)` or sequentially use `connect()`, `submit_recon_job()`, `process_job_and_get_results()`, and `disconnect()`.

Example of how one might use `process_job_and_get_results` (after submitting a job and getting a `job_id`):
```python
# Assuming client is connected and job_id was obtained from submit_recon_job
# if job_id:
#     print(f"Waiting for job {job_id} to complete and receive results...")
#     if client.process_job_and_get_results(job_id):
#         print(f"Job {job_id} completed and results (if any) should be in '{client.client_download_dir}'.")
#     else:
#         print(f"Job {job_id} failed or results could not be processed.")
```
Running the cell above would require the server to actually process the job and send back results, which depends on the server's `RECON_SCRIPT_PATH` being functional for the dummy files.

## 6. Conclusion

This notebook has shown how to use the refactored `ReconClientApp` to programmatically connect to the Recon server, submit jobs, and query server/queue status. For more complex interactions or full job cycles including result downloads, the `run_full_job_cycle` method or a combination of the demonstrated methods can be used.